In [ ]:
import re
from pathlib import Path
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt

# Keep all helper functions the same until generate_analysis

def generate_analysis(real_world_path: Path, model_path: Path, model_name: str) -> str:
    """Generate analysis report comparing real world results with model predictions."""
    # Read markdown files
    with open(real_world_path) as f:
        real_world_content = f.read()
    with open(model_path) as f:
        model_content = f.read()
    
    # Parse standings
    real_standings = parse_markdown_standings(real_world_content)
    model_standings = parse_markdown_standings(model_content)
    
    # Compare standings
    comparison = compare_standings(real_standings, model_standings)
    
    # Generate report
    report = f"## {model_name}\n\n"
    
    for stage in comparison:
        report += f"### {stage}\n\n"
        report += f"Average rank difference: {comparison[stage]['avg_rank_diff']:.2f}\n"
        report += f"Maximum rank difference: {comparison[stage]['max_rank_diff']}\n\n"
        
        report += "| Team | Real Rank | Model Rank | Rank Difference |\n"
        report += "|------|-----------|------------|----------------|\n"
        
        # Sort details by rank difference
        sorted_details = sorted(
            comparison[stage]['details'],
            key=lambda x: (-x['rank_diff'], x['real_rank'])
        )
        
        for detail in sorted_details:
            report += f"| {detail['team']} | {detail['real_rank']} | {detail['model_rank']} | {detail['rank_diff']} |\n"
        
        report += "\n---\n\n"
    
    return report, comparison

def analyze_standings():
    # Get the current working directory and navigate to results folder
    current_dir = Path.cwd()
    if current_dir.name == 'analysis':
        results_dir = current_dir.parent
    else:
        results_dir = current_dir
    analysis_dir = results_dir / 'analysis'
    
    # Create analysis directory if it doesn't exist
    analysis_dir.mkdir(exist_ok=True)
    
    real_world_path = results_dir / '0real-world' / 'README.md'
    model_comparisons = {}
    all_reports = ["# CS2 Match Prediction Standings Analysis\n\n"]
    
    # Analyze each model
    for model_dir in results_dir.iterdir():
        if model_dir.is_dir() and model_dir.name != '0real-world' and not model_dir.name.startswith('.'):
            model_path = model_dir / 'README.md'
            if model_path.exists():
                print(f"Analyzing {model_dir.name}...")
                report, comparison = generate_analysis(real_world_path, model_path, model_dir.name)
                all_reports.append(report)
                model_comparisons[model_dir.name] = comparison
    
    # Save combined report
    with open(analysis_dir / 'analyze-standings.md', 'w') as f:
        f.write('\n'.join(all_reports))
    
    # Generate plots
    plot_rank_differences(model_comparisons, analysis_dir)
    
    print(f"Analysis complete. Results saved in {analysis_dir}")
    return model_comparisons

# Run the analysis
model_comparisons = analyze_standings()

Analyzing deepseek-chat...
Analyzing sabia3...
Analyzing gpt41...
Analyzing claude-sonnet-4...
Analyzing claude-opus-4...
Analyzing gpt-o4-mini...
Analysis complete. Results saved in /Users/lui/oss/cs2-match-prediction/results/analysis
